# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka with Watermarking)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

##### Team grandeInformacion members:
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [1]:
import findspark
findspark.init()
#3a90032bf97eb37c91147f7283c2cec742594ffabf9d3cd7

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Decision-Trees") \
    .master("spark://2e9c357ab096:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 20:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from team_name.spark_utils import SparkUtils
iris_schema = SparkUtils.generate_schema([("Id", "integer"), ("SepalLengthCm", "float"), ("SepalWidthCm","float"), ("PetalLengthCm","float"), ("PetalWidthCm", "float"), ("Species", "string")])

iris_df = spark.read \
                .schema(iris_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/Iris.csv")

iris_df.printSchema()

iris_df.show(5, truncate=False)

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: float (nullable = true)
 |-- SepalWidthCm: float (nullable = true)
 |-- PetalLengthCm: float (nullable = true)
 |-- PetalWidthCm: float (nullable = true)
 |-- Species: string (nullable = true)



+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



In [4]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
label_indexer = StringIndexer(inputCol="Species", outputCol="label")
data_label = label_indexer.fit(iris_df).transform(iris_df)
data_with_features = assembler.transform(data_label).select("label", "features")

In [5]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

In [6]:
print("Original Dataset")
data_with_features.show()

# Print train dataset
print("train set")
train_df.show()

Original Dataset
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[5.09999990463256...|
|  0.0|[4.90000009536743...|
|  0.0|[4.69999980926513...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.5999999046...|
|  0.0|[5.40000009536743...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.4000000953...|
|  0.0|[4.40000009536743...|
|  0.0|[4.90000009536743...|
|  0.0|[5.40000009536743...|
|  0.0|[4.80000019073486...|
|  0.0|[4.80000019073486...|
|  0.0|[4.30000019073486...|
|  0.0|[5.80000019073486...|
|  0.0|[5.69999980926513...|
|  0.0|[5.40000009536743...|
|  0.0|[5.09999990463256...|
|  0.0|[5.69999980926513...|
|  0.0|[5.09999990463256...|
+-----+--------------------+
only showing top 20 rows

train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[4.30000019073486...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.5,2.2999999523...|
|  0.0|[4.599999904

In [7]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [8]:
dt_model = dt.fit(train_df)

# Display model summary
print("Decision Tree model summary:{0}".format(dt_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_058257234ad5, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.449999988079071)
   Predict: 0.0
  Else (feature 2 > 2.449999988079071)
   If (feature 2 <= 4.8500001430511475)
    If (feature 3 <= 1.6500000357627869)
     Predict: 1.0
    Else (feature 3 > 1.6500000357627869)
     If (feature 0 <= 5.950000047683716)
      Predict: 1.0
     Else (feature 0 > 5.950000047683716)
      Predict: 2.0
   Else (feature 2 > 4.8500001430511475)
    If (feature 3 <= 1.75)
     If (feature 2 <= 4.950000047683716)
      Predict: 1.0
     Else (feature 2 > 4.950000047683716)
      If (feature 3 <= 1.550000011920929)
       Predict: 2.0
      Else (feature 3 > 1.550000011920929)
       Predict: 1.0
    Else (feature 3 > 1.75)
     Predict: 2.0



In [9]:
# Use the trained model to make predictions on the test data
predictions = dt_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,3.4000000953...|       0.0|
|[5.0,3.5,1.299999...|       0.0|
|[5.09999990463256...|       0.0|
|[5.19999980926513...|       0.0|
|[5.40000009536743...|       0.0|
|[5.69999980926513...|       1.0|
|[5.80000019073486...|       1.0|
|[5.80000019073486...|       1.0|
|[6.09999990463256...|       1.0|
|[6.09999990463256...|       1.0|
|[6.19999980926513...|       1.0|
|[6.30000019073486...|       1.0|
|[6.69999980926513...|       1.0|
|[4.90000009536743...|       1.0|
|[6.0,3.0,4.800000...|       2.0|
|[6.09999990463256...|       2.0|
|[6.19999980926513...|       2.0|
|[6.30000019073486...|       2.0|
|[6.30000019073486...|       2.0|
|[6.40000009536743...|       2.0|
+--------------------+----------+
only showing top 20 rows



In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")    

Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545


In [11]:
from pyspark.ml.classification import LinearSVC, OneVsRest

# LinearSVC
lsvc = LinearSVC(maxIter=10, regParam=0.1, labelCol="label", featuresCol="features")

# OneVsRest
ovr = OneVsRest(classifier=lsvc, labelCol="label", featuresCol="features")

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

# Train model
ovr_model = ovr.fit(train_df)

ovr_predictions = ovr_model.transform(test_df)

accuracy_ovr = evaluator.evaluate(ovr_predictions, {evaluator.metricName: "accuracy"})
print(f"Accuracy Score (OneVsRest + LinearSVC): {accuracy_ovr}")

precision_ovr = evaluator.evaluate(ovr_predictions, {evaluator.metricName: "weightedPrecision"})
print(f"Precision Score (OneVsRest + LinearSVC): {precision_ovr}")

recall_ovr = evaluator.evaluate(ovr_predictions, {evaluator.metricName: "weightedRecall"})
print(f"Recall Score (OneVsRest + LinearSVC): {recall_ovr}")

f1_ovr = evaluator.evaluate(ovr_predictions, {evaluator.metricName: "f1"})
print(f"F1 Score (OneVsRest + LinearSVC): {f1_ovr}")

F1 Score (OneVsRest + LinearSVC): 0.8142857142857143
Accuracy Score (OneVsRest + LinearSVC): 0.8181818181818182
Precision Score (OneVsRest + LinearSVC): 0.8278236914600551
Recall Score (OneVsRest + LinearSVC): 0.8181818181818181


In [16]:
if f1 > f1_ovr:
    print(f"Decision Tree performed better based on F1 Score.\n Decision Tree F1 Score: {f1}\n OneVsRest + LinearSVC F1 Score: {f1_ovr}")
else:
    print(f"OneVsRest + LinearSVC performed better based on F1 Score.\n OneVsRest + LinearSVC F1 Score: {f1_ovr}\n Decision Tree F1 Score: {f1}")

Decision Tree performed better based on F1 Score.
 Decision Tree F1 Score: 0.9545454545454545
 OneVsRest + LinearSVC F1 Score: 0.8142857142857143
